<a href="https://colab.research.google.com/github/FeraMaks/test/blob/main/NN_ShaftTraectory_2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import pandas as pd
from numpy.core.memmap import uint8

from google.colab import drive

In [2]:
#device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


##Загрузка данных

Эллиптические графики

In [4]:
# Советую воспользоваться циклом for, чтобы не писать множество однотипных строк кода

data1 = pd.read_excel(open('/content/drive/MyDrive/work/100_000.xlsx', 'rb'))
data1 = pd.DataFrame(data1)
data1 = pd.DataFrame.to_numpy(data1)
data1 = np.rot90(data1)
data1.shape

(6, 20000)

In [5]:
data2 = pd.read_excel(open('/content/drive/MyDrive/work/90_000.xlsx', 'rb'))
data2 = pd.DataFrame(data2)
data2 = pd.DataFrame.to_numpy(data2)
data2 = np.rot90(data2)
data2.shape

(6, 20000)

In [6]:
data3 = pd.read_excel(open('/content/drive/MyDrive/work/110_000.xlsx', 'rb'))
data3 = pd.DataFrame(data3)
data3 = pd.DataFrame.to_numpy(data3)
data3 = np.rot90(data3)
data3.shape

(6, 20000)

In [7]:
data4 = pd.read_excel(open('/content/drive/MyDrive/work/120_000.xlsx', 'rb'))
data4 = pd.DataFrame(data4)
data4 = pd.DataFrame.to_numpy(data4)
data4 = np.rot90(data4)
data4.shape

(6, 20000)

In [8]:
data5 = pd.read_excel(open('/content/drive/MyDrive/work/001_19-12-05_1704_021_000.xlsx', 'rb'))
data5 = pd.DataFrame(data5)
data5 = pd.DataFrame.to_numpy(data5)
data5 = np.rot90(data5)
data5.shape

(6, 20000)

Метастабильные графики

In [9]:
data6 = pd.read_excel(open('/content/drive/MyDrive/work/001_19-12-05_1739_005-290_222.xlsx', 'rb'))
data6 = pd.DataFrame(data6)
data6 = pd.DataFrame.to_numpy(data6)
data6 = np.rot90(data6)
data6.shape

(6, 20000)

In [10]:
data7 = pd.read_excel(open('/content/drive/MyDrive/work/001_19-12-05_1739_006_222.xlsx', 'rb'))
data7 = pd.DataFrame(data7)
data7 = pd.DataFrame.to_numpy(data7)
data7 = np.rot90(data7)
data7.shape

(6, 20000)

In [11]:
data8 = pd.read_excel(open('/content/drive/MyDrive/work/001_19-12-05_1740_007_222.xlsx', 'rb'))
data8 = pd.DataFrame(data8)
data8 = pd.DataFrame.to_numpy(data8)
data8 = np.rot90(data8)
data8.shape

(6, 20000)

In [12]:
data9 = pd.read_excel(open('/content/drive/MyDrive/work/001_19-12-05_1740_009_222.xlsx', 'rb'))
data9 = pd.DataFrame(data9)
data9 = pd.DataFrame.to_numpy(data9)
data9 = np.rot90(data9)
data9.shape

(6, 20000)

In [13]:
data10 = pd.read_excel(open('/content/drive/MyDrive/work/001_19-12-05_1741_010_222.xlsx', 'rb'))
data10 = pd.DataFrame(data10)
data10 = pd.DataFrame.to_numpy(data10)
data10 = np.rot90(data10)
data10.shape

(6, 20000)

##Подготовка датасета

In [14]:
#parameters
data_number = 10 # количество файлов с графиками
num_samples = 10 # количество сэмплов в одном графике
point_number = 2000 # количество точек в одном графике
num_graphycs = 3 # количество графиков
num_grahp0 = 3 # кол-во эллиптических графиков
num_grahp2 = 3 # кол-во метастабильных графиков
inputs = []

In [15]:
y0 = np.zeros((num_grahp0*num_samples*5,1))
y2 = np.ones((num_grahp2*num_samples*5,1))
y2 = y2
y = np.concatenate((y0, y2))
#for i in range (5):
#  y = np.concatenate((y, y))
y, y.shape

(array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
      

In [16]:
data_x = np.concatenate((data1,data2,data3,data4,data5,data6,data7,data8,data9,data10))
data_x.shape

(60, 20000)

In [17]:
for j in range (data_number*num_graphycs):
  for i in range (num_samples):
    inputs.append(np.concatenate((data_x[j*2,i*point_number:i*point_number+point_number], data_x[j*2+1,i*point_number:i*point_number+point_number])))
inputs = np.concatenate(inputs, axis=0)
inputs = np.reshape(inputs, (data_number*num_graphycs*num_samples,point_number*2))
# Тип данных  в нейросетях - float32. Можно и float64, но тогда модель нужно делать с таким типом данных.
#x = x.astype('float32')
inputs.shape

(300, 4000)

In [18]:
xy = np.concatenate((y,inputs), axis=1)
xy[0,:]

array([   0.      , -182.154261, -182.813766, ...,  -46.86475 ,
        -44.582413,  -42.288928])

In [19]:
xy = sorted(xy, key=lambda x: random.random())

In [20]:
xy = np.asarray(xy)
xy.shape

(300, 4001)

In [21]:
x, y = xy[:,1:], xy[:,0]
x = x.astype('float32')
y = y.astype('int64')
x.shape, y.shape

((300, 4000), (300,))

In [22]:
# Индексация классов начиается с нуля. То есть первый класс - 0, второй класс - 1. Тип данных разметки - int64.
# Размерность у разметки должна быть одна

In [23]:
# Тут и далее уже очень много ошибок, покажу просто как правильно с комментариями.

class ShaftDataset(Dataset):
    def __init__(self, x, y, mode):
        # Проверка выборки
        if mode == 'train':
            self.x = x[:260]
            self.y = y[:260]
        elif mode == 'val':
            self.x = x[260:]
            self.y = y[260:]
            
        # Разбиение сделано по вашему примеру, но обращаю внимание,
        # что так в тренировочной выборке всего один пример со 2 классом. А в тестовой - только 2 класс/
        # Поэтому нужно как то примеры скомпоновать по-другому

    def __getitem__(self, index):
        return self.x[index], self.y[index]
  
    def __len__(self):
        return len(self.y)

In [24]:
# Создаем два отдельных датасета и их будем подавать в Loader.
# Изначально вы пытались сделать два среза по одному датасету и подавили в Loader. Так работать не будет

train_dataset = ShaftDataset(x, y, 'train')
test_dataset = ShaftDataset(x, y, 'val')


In [25]:
train_dataset[0]

(array([ -4.110191,  -4.534195,  -4.953076, ..., -21.964207, -22.143084,
        -22.317085], dtype=float32), 0)

Hyper parameters

In [26]:
input_size = x.shape[1]
hidden_size = 32 # Попробуйте для начала поменьше
num_classes = 2
num_epochs = 1000
batch_size = 15 # Bath size не может быть больше размера датасета (у вас 30 примеров)
lr = 0.001 # learning rate

In [27]:
# Loader генерирует тензоры следующего размера -
# (batch_size (размер батча - кол-во примеров в батче), m (кол-во признаков))
# Таким образом, размерностей 2, а не 4

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [28]:
# Размеры данных в датасетах

features, labels = train_dataset.x, train_dataset.y
print(features.shape, labels.shape)
features, labels = test_dataset.x, test_dataset.y
print(features.shape, labels.shape)
#samples = torch.from_numpy(samples)
#labels = torch.from_numpy(labels)

(260, 4000) (260,)
(40, 4000) (40,)


#Создание нейронной сети

In [29]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    return out


In [30]:
model = NeuralNet(input_size, hidden_size, num_classes)

# Модель также перемещаем в нужное место
model = model.to(device)

Loss and optimizer

In [31]:
L = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#Training loop

In [32]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (features, labels) in enumerate(train_loader):
    features = features.to(device)
    labels = labels.to(device)

    #forward path
    outputs = model(features) # Подаем признаки на вход нейросети
    loss = L(outputs, labels)

    #backward path
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 7 == 0:
       print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')


epoch 1 / 1000, step 7/18, loss = 18.7549
epoch 1 / 1000, step 14/18, loss = 0.4942
epoch 2 / 1000, step 7/18, loss = 0.0000
epoch 2 / 1000, step 14/18, loss = 2.2362
epoch 3 / 1000, step 7/18, loss = 3.5404
epoch 3 / 1000, step 14/18, loss = 0.0000
epoch 4 / 1000, step 7/18, loss = 0.0000
epoch 4 / 1000, step 14/18, loss = 10.1191
epoch 5 / 1000, step 7/18, loss = 0.0022
epoch 5 / 1000, step 14/18, loss = 0.0000
epoch 6 / 1000, step 7/18, loss = 0.0000
epoch 6 / 1000, step 14/18, loss = 0.0000
epoch 7 / 1000, step 7/18, loss = 0.0000
epoch 7 / 1000, step 14/18, loss = 0.0000
epoch 8 / 1000, step 7/18, loss = 0.0000
epoch 8 / 1000, step 14/18, loss = 0.0000
epoch 9 / 1000, step 7/18, loss = 0.0000
epoch 9 / 1000, step 14/18, loss = 0.0000
epoch 10 / 1000, step 7/18, loss = 0.0000
epoch 10 / 1000, step 14/18, loss = 0.0000
epoch 11 / 1000, step 7/18, loss = 0.0000
epoch 11 / 1000, step 14/18, loss = 0.0000
epoch 12 / 1000, step 7/18, loss = 0.0000
epoch 12 / 1000, step 14/18, loss = 0.0

In [33]:
outputs, labels

(tensor([[  739.0110,  -359.7642],
         [  -30.9535,    58.0084],
         [ 1751.5850, -4129.3096],
         [   35.3523,   111.9118],
         [   27.6965,    78.7408]], grad_fn=<AddmmBackward0>),
 tensor([0, 1, 0, 1, 1]))

#Test

In [34]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for features, labels in test_loader:
    features = features.to(device)
    labels = labels.to(device)
    outputs = model(features)
    print(outputs)
    #value, index
    _, predictions = torch.max(outputs, 1)
    n_samples += labels.shape[0]
    n_correct += (predictions == labels).sum().item()

tensor([[  607.7876,  -391.9648],
        [  898.9504, -1165.2142],
        [   23.8828,   118.1226],
        [  -19.4923,   110.2719],
        [  456.2417,  -338.7886],
        [  -16.7248,   102.1257],
        [  181.7005,  -266.9938],
        [  499.8809,  -397.1997],
        [  707.5892,  -905.6423],
        [   43.2559,     7.7117],
        [  558.4290,  -999.7791],
        [  535.0967, -1067.9081],
        [  -13.4056,    70.6706],
        [  551.7296, -1233.0510],
        [   12.3883,   129.4810]])
tensor([[   75.4558,   -79.5103],
        [ 1287.5945, -1047.2839],
        [  500.8370,  -166.2628],
        [ 1611.6820, -1919.0743],
        [  473.1199,   171.3976],
        [  635.6432,  -229.1887],
        [  -36.4442,    25.7312],
        [   14.6253,   131.4946],
        [  512.9720,  -641.1506],
        [ 3012.7361, -2445.6467],
        [  -34.8559,    89.6291],
        [ 2424.7861, -3066.7219],
        [  -14.6261,   110.3412],
        [  237.8718,  -272.2787],
        [  -2

In [35]:
  acc = 100.0 * n_correct / n_samples
  print(f'accuracy = {acc} %')

accuracy = 95.0 %
